In [ ]:
# %matplotlib qt
# %matplotlib ipympl

In [ ]:
## Imports
print("Importing...")
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import cv2
import easyocr

from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.widgets import Slider


In [ ]:
VIDEO_PATH = "Videos/Full_font1.avi"
# VIDEO_PATH = None
if VIDEO_PATH is None:
    input_path = ''
    while input_path == '':
        input_path = input(f"Input video path: ")
    VIDEO_PATH = input_path

RECOGNIZABLE_VARIABLES = [{'name': 'Viscosity'}, {'name': 'Temperature'}]

CAP = cv2.VideoCapture(VIDEO_PATH)
FPS = int(CAP.get(cv2.CAP_PROP_FPS))
LENTH = int(CAP.get(cv2.CAP_PROP_FRAME_COUNT) / FPS)
CAP.set(cv2.CAP_PROP_POS_FRAMES, 0)
_, START_FRAME = CAP.read()


In [ ]:
## Preprocess
class ImageProcessor:

    def __init__(self, blur=1):
        self.blur = blur

    def __call__(self, frame):
        frame = cv2.blur(frame, (self.blur, self.blur))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = cv2.bitwise_not(frame)
        return frame


fig, ax = plt.subplots()
fig.set_size_inches(5, 5)
fig.subplots_adjust(left=0.25, right=1, bottom=0.25, top=1, hspace=0, wspace=0)

PROCESSOR = ImageProcessor()
PLOT = ax.imshow(PROCESSOR(START_FRAME), cmap='binary')

ax_time_slider = fig.add_axes([0.25, 0.1, 0.65, 0.03])
TIME_slider = Slider(
    ax=ax_time_slider,
    label='Time',
    valmin=0,
    valmax=LENTH,
    valinit=0,
    valstep=1,
)

ax_blur_slider = fig.add_axes([0.1, 0.25, 0.03, 0.6])
BLUR_slider = Slider(
    ax=ax_blur_slider,
    orientation='vertical',
    label='Blur',
    valmin=PROCESSOR.blur,
    valmax=20,
    valinit=1,
    valstep=1,
)


def update(val):
    time = TIME_slider.val
    global PROCESSOR
    PROCESSOR.blur = BLUR_slider.val

    CAP.set(cv2.CAP_PROP_POS_FRAMES, int(FPS * time))
    _, frame = CAP.read()
    frame = PROCESSOR(frame)

    PLOT.set_data(frame)
    PLOT.autoscale()

    fig.canvas.draw_idle()


TIME_slider.on_changed(update)
BLUR_slider.on_changed(update)
print('Configurate image processing')
plt.show()

In [ ]:
## Selection
print(f'Blur value= {PROCESSOR.blur}')


def strict(image, x, y, w, h):
    return image[y:y + h, x:x + w]


for variable in RECOGNIZABLE_VARIABLES:
    roi_frame = PROCESSOR(START_FRAME)
    roi_frame = cv2.bitwise_not(roi_frame)
    current_slice = cv2.selectROI(
        f"Select {variable['name']}",
        roi_frame,
        fromCenter=False,
        showCrosshair=True,
    )
    variable['slice'] = current_slice
cv2.destroyAllWindows()

# Result plots
fig, axes = plt.subplots(nrows=len(RECOGNIZABLE_VARIABLES))
if not isinstance(axes, np.ndarray): axes = [axes]
fig.set_size_inches(3, 1 * len(RECOGNIZABLE_VARIABLES))
fig.subplots_adjust(left=0, right=1, bottom=0.0, top=1, hspace=0, wspace=0)
start_slices = [
    strict(PROCESSOR(START_FRAME), *variable['slice'])
    for variable in RECOGNIZABLE_VARIABLES
]
for i in range(len(RECOGNIZABLE_VARIABLES)):
    axes[i].set_axis_off()
    axes[i].imshow(start_slices[i], cmap='binary')
plt.show()

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(
    [
        {'t':1,'g':3},
        {'t':2,'g':5}
    ]
    
)

In [ ]:
# Recognize
print('Starting recognizer...')
reader = easyocr.Reader(['en'])

input_fps = input('Input number of frames per second: ')
try:
    read_fps = int(input_fps)
except:
    read_fps = 1

print('Recognizing:')
diap = tqdm(iterable=range(0, FPS * LENTH, int(FPS / read_fps)))

data = []
for i_frame in diap:
    CAP.set(cv2.CAP_PROP_POS_FRAMES, i_frame)
    _, frame = CAP.read()
    i_text = {}
    for variable in RECOGNIZABLE_VARIABLES:
        selection = strict(frame, *variable['slice'])
        selection = PROCESSOR(selection)

        raw_values = [value for _, value, _ in reader.readtext(selection)]
        # values = correct(raw_values)
        result= variable.copy()
        
        i_text[variable['name']]=raw_values[0]

    data.append(i_text)
pd.DataFrame(data)

In [ ]:
raw_value = data[0][0]

In [ ]:
raw_value

In [ ]:
int('123.3')

In [ ]:
if not 7988 // 1000: print('asdf')


In [ ]:
min_val = 12


In [ ]:
10 <= min_val if min_val is not None else True

In [ ]:
import re

In [ ]:
class ValueCorrector:

    def __init__(self, processor, reader):
        self._processor = processor
        self._reader = reader

    def pattern_check(self, value: str, re_rule, min_val, max_val):
        value = value.replace(',', '.')
        one_check = len(re.findall(re_rule, value)) == 1
        try:
            result = float(value[0])
        except ValueError:
            return None
        min_check = value <= min_val if re_rule is not None else True
        max_check = value >= max_val if re_rule is not None else True

        result = value if one_check and min_check and max_check else None
        return result

    def correct(self, raw_value, rules: dict):
        one = len(raw_value) == 1
        result = self.pattern_check(raw_value[0], **rules)
        if one and (result is not None): return result

        try:
            result = int(raw_value[0])
            value = self._int_check(result)
        except ValueError:
            self._value_check(result)

        self._process

        return value

    @staticmethod
    def _int_check(value):
        if value <= 1000: return value

    def _value_check(self):
        ...

    def _combine(self):

        if parts == 2:
            raw_result = '.'.join(values)

        elif parts == 3:
            raw_result = f'{values[0]}.{values[2]}'

        try:
            result = float(raw_result)
        except:
            print('error', raw_result)
            result = None
        return result

    @staticmethod
    def _process(image, ):
        ...


In [ ]:
a = ValueCorrector(raw_value)
a._combine()